In [4]:
import torch
from torchvision.models import resnet18
from torchvision import transforms
from PIL import Image
from datetime import datetime
import os

# Загрузка модели ResNet18
model = resnet18(pretrained=False)  # Создаём модель
num_classes = 10  # Количество классов в вашей задаче
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)  # Настраиваем последний слой

model_path = 'resnet18_biology.pth'  # Укажите путь к вашей сохранённой модели
model.load_state_dict(torch.load(model_path))  # Загружаем веса
model.eval()

# Устройство выполнения
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Преобразования для изображения
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Папка для журнала с изображениями
log_dir = "classification_log"
os.makedirs(log_dir, exist_ok=True)

# Функция классификации
def classify_image(image_path):
    try:
        # Открываем изображение
        image = Image.open(image_path).convert('RGB')

        # Преобразуем изображение
        input_tensor = transform(image).unsqueeze(0).to(device)

        # Классификация
        with torch.no_grad():
            outputs = model(input_tensor)
            _, predicted = torch.max(outputs, 1)
        
        # Имя класса (замените `class_names` на ваш список классов)
        class_names = ["Собака", "Лошадь", "Слон", "Бабочка", "Курица", "Кошка", "Корова", "Овца", "Паук", "Белка"]
        class_name = class_names[predicted.item()]

        # Метка времени
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

        # Сохраняем изображение в журнал
        saved_image_path = os.path.join(log_dir, f"{timestamp}_{class_name}.jpg")
        image.save(saved_image_path)

        # Сохраняем запись в журнал
        with open(os.path.join(log_dir, "log.txt"), "a") as log:
            log.write(f"{timestamp}, {class_name}, {saved_image_path}\n")

        print(f"Изображение сохранено: {saved_image_path}, Класс: {class_name}")
    except Exception as e:
        print(f"Ошибка при обработке изображения: {e}")

# Классифицируем изображение photo.jpg
classify_image("photo.jpg")


Изображение: photo.jpg, Класс: Паук, Время: 2024-12-25 11:37:34


C:\Users\tseli\AppData\Local\Temp\ipykernel_13224\2160936303.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))  # Загружаем 